# TF в Colab

In [0]:
%config IPCompleter.greedy=True  # более лучший автокомплит

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

NotFoundError: dlopen(/Users/nadezdaoskina/.pyenv/versions/3.10.9/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '__ZN3tsl8internal10LogMessageC1EPKcii'

In [ ]:
tf.reset_default_graph()
a = tf.placeholder(tf.float32, (2, 2))
b = tf.Variable(tf.ones((2, 2)))
c = a @ b

In [ ]:
print(c)

In [ ]:
print(list(tf.get_default_graph().get_operations()[-1].inputs))
print(list(tf.get_default_graph().get_operations()[-1].outputs))

In [0]:
s = tf.InteractiveSession()

In [ ]:
s.run(tf.global_variables_initializer())
s.run(c, feed_dict={a: np.ones((2, 2))})

In [0]:
s.close()

# Оптимизация

In [0]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32, trainable=True)
f = x ** 2

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f, var_list=[x])

In [ ]:
tf.trainable_variables()

In [ ]:
with tf.Session() as s:  # сессия сама закроется
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f])
        print(curr_x, curr_f)

# TensorBoard логирование

In [0]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [0]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all()

In [0]:
with tf.Session() as s:
  summary_writer = tf.summary.FileWriter("logs/1", s.graph)
  s.run(tf.global_variables_initializer())
  for i in range(10):
      _, curr_summaries = s.run([step, summaries])
      summary_writer.add_summary(curr_summaries, i)
      summary_writer.flush()

Локально можно запустить  `tensorboard --logdir=./logs` в консоли.

А вот так можно запустить прямо в Google Colab:

In [0]:
! wget https://raw.githubusercontent.com/ZEMUSHKA/skillfactory-dl/master/setup_google_colab.py -O setup_google_colab.py -q
import setup_google_colab

# запускаем сервер tensorboard в фоне
import os
os.system("tensorboard --logdir=./logs --host 0.0.0.0 --port 6006 &")

# открываем порт из Google Colab
setup_google_colab.expose_port_on_colab(6006)

# Обучение линейной модели

In [ ]:
# модельные данные
N = 1000
D = 3
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T)

In [ ]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D), name="features")
target = tf.placeholder(tf.float32, shape=(None, 1), name="target")

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights


loss = tf.reduce_mean((target - predictions) ** 2, name="loss")

print(target.shape, predictions.shape, loss.shape)

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [0]:
# производные это часть графа
tf.get_default_graph().get_operations()

In [ ]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss)

In [ ]:
# результат обучения
curr_weights.T

In [ ]:
# правильные веса
w.T

# Обучение в Keras

In [ ]:
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
print(keras.__version__)

In [ ]:
K.clear_session()  # очищает граф и перезапускает сессию

model = M.Sequential()
model.add(L.Dense(1, use_bias=False, input_shape=(D,)))
model.summary()

In [0]:
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

In [ ]:
model.fit(x, y, batch_size=8, epochs=10, callbacks=[keras.callbacks.TensorBoard("./logs/keras")])

In [ ]:
s = K.get_session()
s.run(model.weights)